In [1]:
import pandas as pd
import numpy as np

### Linkedin Data Scraping Code

In [2]:
from selenium import webdriver
import time

In [3]:
#Import Packages
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [4]:
url = "https://www.linkedin.com/jobs/search/?currentJobId=3256703076&geoId=102713980&keywords=engineering&location=India&refresh=true"

In [5]:
driver = webdriver.Chrome(executable_path="chromedriver.exe")
driver.get(url)

C:\Users\Anand\AppData\Local\Temp\ipykernel_9520\1118520676.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="chromedriver.exe")


In [6]:
# store all links here
links = []

In [7]:
try:
    for page in range(2,26):
        time.sleep(2)
        job_block = driver.find_element(By.CLASS_NAME,"jobs-search-results-list")
        jobs_list= job_block.find_elements(By.CSS_SELECTOR, '.jobs-search-results__list-item')

        for job in jobs_list:
            all_links = job.find_elements(By.TAG_NAME,'a')
            
            for a in all_links:
                if str(a.get_attribute('href')).startswith("https://www.linkedin.com/jobs/view") and a.get_attribute('href') not in links: 
                    links.append(a.get_attribute('href'))
                else:
                    pass
            
            #scroll down for each job element
            
            driver.execute_script("arguments[0].scrollIntoView();", job)
        
        # go to next page:

        driver.find_element(By.XPATH,f"//button[@aria-label='Page {page}']").click()
        time.sleep(3)
        
except:
    pass


In [10]:
location = []
designation = []
company_name = []
industry_type = []
employees_count = []
linkedin_followers = []
involvement = []
level = []
total_applicants = []

In [11]:
i = 0
# Visit each link one by one to scrape the information
for i in range(450):
    try:
        driver.get(links[i])
        i=i+1
        time.sleep(3)

    except:
        pass
    
    # collect data like company,designation,location,applicant,etc from each job link
    
    company_data = driver.find_elements(By.CLASS_NAME,"p5")

    for data in company_data:
        try:
            designation.append(data.find_element(By.TAG_NAME,"h1").text)
        except:
            designation.append("NA")
        try:
            location.append(data.find_element(By.CLASS_NAME,"jobs-unified-top-card__bullet").text)
        except:
            location.append('NA')
        try:
            company_name.append(data.find_element(By.CLASS_NAME,"jobs-unified-top-card__company-name").text)
        except:
            company_name.append('NA')
        try:
            involvement.append(data.find_element(By.CLASS_NAME,"jobs-unified-top-card__job-insight").text)
        except:
            involvement.append('NA')
        try:
            industry_type.append(data.find_element(By.TAG_NAME,"ul").text)
        except:
            industry_type.append('NA')
        try:
            total_applicants.append(data.find_element(By.CLASS_NAME,"jobs-unified-top-card__applicant-count").text)
        except:
            total_applicants.append('NA')
    
    #to find the linkedin follower of the company:
            
    driver.execute_script("window.scrollTo(0,700)")
    time.sleep(2)
    
    company_follower = driver.find_elements(By.CLASS_NAME,"jobs-company__box")
    
    for j in company_follower:
        try:
            linkedin_followers.append(j.find_element(By.CSS_SELECTOR,".artdeco-entity-lockup__subtitle").text)
        except:
            linkedin_followers.append('NA')

### Data Cleaning and Processing

In [14]:
df = pd.DataFrame({"designation":designation,"location":location,"company_name":company_name,"involvement":involvement,"industry_type":industry_type,"total_applicants":total_applicants})

In [15]:
df1 = pd.DataFrame({"linkedin_followers":linkedin_followers})

In [16]:
df.to_csv("PythonProject.csv")

In [17]:
df1.to_csv("followers.csv")

In [2]:
df2 = pd.read_csv("PythonProject.csv")

In [ ]:
df2.duplicated().value_counts()

In [6]:
df2[["a","b","c","d","e"]] = df2.industry_type.str.split("\n",expand=True)

In [18]:
df2.drop(["industry_type"],axis=1,inplace=True)

In [19]:
df2[["employees_count","industry_type"]] = df2.b.str.split("·",expand=True)

In [21]:
df2.drop(["a","b","c","d","e"],axis=1,inplace=True)

In [24]:
df2.drop(["Unnamed: 0"],axis=1,inplace=True)

In [25]:
df2

,designation,location,company_name,involvement,total_applicants,employees_count,industry_type
0,Data Analyst For 5 Day Working USA MNC In Gurg...,"Gurgaon, Haryana, India",Capital Placement Services Gurgaon .,Full-time · Entry level,34 applicants,1-10 employees,Human Resources Services
1,Zoomcar - Program Manager - Process Excellence,"Bengaluru, Karnataka, India",Zoomcar,Full-time · Entry level,50 applicants,201-500 employees,"Technology, Information and Internet"
2,Data Analyst (OpEx Engineering),"Mumbai, Maharashtra, India",Fynd (Shopsense Retail Technologies Ltd.),Full-time · Entry level,39 applicants,"501-1,000 employees",Internet Marketplace Platforms
3,UCC Quality Monitoring Developer/Analyst,"Bengaluru, Karnataka, India",Bosch Global Software Technologies,Full-time · Associate,65 applicants,"10,001+ employees",IT Services and IT Consulting
4,Associate Systems Analyst,"Pune, Maharashtra, India",Allscripts India,Full-time · Entry level,52 applicants,"1,001-5,000 employees",IT Services and IT Consulting
...,...,...,...,...,...,...,...
456,Product Marketing - General Manager,"Noida, Uttar Pradesh, India",Paytm,Full-time · Executive,200 applicants,"10,001+ employees",Financial Services
457,Principal Software Dev Engr,"Bengaluru, Karnataka, India",Tesco Technology,Full-time,10 applicants,"1,001-5,000 employees",Retail
458,Associate Consultant,"Noida, Uttar Pradesh, India",Huron,Full-time,75 applicants,"1,001-5,000 employees",Business Consulting and Services
459,Simulation Engineering - SPDM,"Bengaluru, Karnataka, India",Bosch Global Software Technologies,Full-time · Associate,36 applicants,"10,001+ employees",IT Services and IT Consulting


In [36]:
df2.to_csv("Linkedin.csv")

### Table Making

In [27]:
#job table
Job = df2[["designation","location"]]

In [28]:
Job

,designation,location
0,Data Analyst For 5 Day Working USA MNC In Gurg...,"Gurgaon, Haryana, India"
1,Zoomcar - Program Manager - Process Excellence,"Bengaluru, Karnataka, India"
2,Data Analyst (OpEx Engineering),"Mumbai, Maharashtra, India"
3,UCC Quality Monitoring Developer/Analyst,"Bengaluru, Karnataka, India"
4,Associate Systems Analyst,"Pune, Maharashtra, India"
...,...,...
456,Product Marketing - General Manager,"Noida, Uttar Pradesh, India"
457,Principal Software Dev Engr,"Bengaluru, Karnataka, India"
458,Associate Consultant,"Noida, Uttar Pradesh, India"
459,Simulation Engineering - SPDM,"Bengaluru, Karnataka, India"


In [29]:
#final Job table completed in Excel
Job.to_csv("Job_table.csv")

In [15]:
# Company Table Making By Using cpnt and and Cfo Csv In Excel
cpn= df2[["company_name","industry_type","employees_count"]]

In [49]:
Company = Company.drop_duplicates()

In [17]:
cpn.to_csv("cpnt.csv")

In [31]:
df3 = pd.read_csv("followers.csv")

In [9]:
f = df3[["linkedin_followers"]]

In [11]:
f.to_csv("cfo.csv")

In [33]:
Details = df2[["involvement","total_applicants"]]

In [34]:
Details

,involvement,total_applicants
0,Full-time · Entry level,34 applicants
1,Full-time · Entry level,50 applicants
2,Full-time · Entry level,39 applicants
3,Full-time · Associate,65 applicants
4,Full-time · Entry level,52 applicants
...,...,...
456,Full-time · Executive,200 applicants
457,Full-time,10 applicants
458,Full-time,75 applicants
459,Full-time · Associate,36 applicants


In [35]:
Details.to_csv("Details_Table.csv")

In [18]:
df =  pd.read_csv("company_table.csv")

In [31]:
df.shape

(372, 4)

In [21]:
df.dropna(axis=0,inplace=True)

In [23]:
df.isnull().sum()

company_name          0
industry_type         0
employees_count       0
linkedin_followers    0
dtype: int64

In [25]:
df.drop_duplicates(inplace=True)

In [37]:
df["company_name"].duplicated().value_counts()

False    259
Name: company_name, dtype: int64

In [34]:
df = df.drop_duplicates(subset="company_name")

In [35]:
df.shape

(259, 4)

In [41]:
df.to_csv("Final_company_table.csv",index=False)

In [42]:
pd.read_csv("Final_company_table.csv")

,company_name,industry_type,employees_count,linkedin_followers
0,Capital Placement Services Gurgaon .,Human Resources Services,1-10 employees,"14,205 followers"
1,Zoomcar,"Technology, Information and Internet",201-500 employees,"46,150 followers"
2,Fynd (Shopsense Retail Technologies Ltd.),Internet Marketplace Platforms,"501-1,000 employees","37,670 followers"
3,Bosch Global Software Technologies,IT Services and IT Consulting,"10,001+ employees","794,635 followers"
4,Allscripts India,IT Services and IT Consulting,"1,001-5,000 employees","10,394 followers"
...,...,...,...,...
254,LambdaTest,Software Development,201-500 employees,"19,305 followers"
255,Infinity Solutions,IT Services and IT Consulting,201-500 employees,"7,449 followers"
256,Primary Healthtech,Medical Equipment Manufacturing,11-50 employees,"1,949 followers"
257,Tesco Technology,Retail,"1,001-5,000 employees","13,984 followers"


In [43]:
df2 = pd.read_csv("Linkedin.csv")

In [44]:
df2

,Unnamed: 0,designation,location,company_name,involvement,total_applicants,employees_count,industry_type
0,0,Data Analyst For 5 Day Working USA MNC In Gurg...,"Gurgaon, Haryana, India",Capital Placement Services Gurgaon .,Full-time · Entry level,34 applicants,1-10 employees,Human Resources Services
1,1,Zoomcar - Program Manager - Process Excellence,"Bengaluru, Karnataka, India",Zoomcar,Full-time · Entry level,50 applicants,201-500 employees,"Technology, Information and Internet"
2,2,Data Analyst (OpEx Engineering),"Mumbai, Maharashtra, India",Fynd (Shopsense Retail Technologies Ltd.),Full-time · Entry level,39 applicants,"501-1,000 employees",Internet Marketplace Platforms
3,3,UCC Quality Monitoring Developer/Analyst,"Bengaluru, Karnataka, India",Bosch Global Software Technologies,Full-time · Associate,65 applicants,"10,001+ employees",IT Services and IT Consulting
4,4,Associate Systems Analyst,"Pune, Maharashtra, India",Allscripts India,Full-time · Entry level,52 applicants,"1,001-5,000 employees",IT Services and IT Consulting
...,...,...,...,...,...,...,...,...
456,456,Product Marketing - General Manager,"Noida, Uttar Pradesh, India",Paytm,Full-time · Executive,200 applicants,"10,001+ employees",Financial Services
457,457,Principal Software Dev Engr,"Bengaluru, Karnataka, India",Tesco Technology,Full-time,10 applicants,"1,001-5,000 employees",Retail
458,458,Associate Consultant,"Noida, Uttar Pradesh, India",Huron,Full-time,75 applicants,"1,001-5,000 employees",Business Consulting and Services
459,459,Simulation Engineering - SPDM,"Bengaluru, Karnataka, India",Bosch Global Software Technologies,Full-time · Associate,36 applicants,"10,001+ employees",IT Services and IT Consulting


In [49]:
df2.shape

(461, 8)

In [50]:
df2.dropna(axis=0,inplace=True)

In [51]:
df2

,Unnamed: 0,designation,location,company_name,involvement,total_applicants,employees_count,industry_type
0,0,Data Analyst For 5 Day Working USA MNC In Gurg...,"Gurgaon, Haryana, India",Capital Placement Services Gurgaon .,Full-time · Entry level,34 applicants,1-10 employees,Human Resources Services
1,1,Zoomcar - Program Manager - Process Excellence,"Bengaluru, Karnataka, India",Zoomcar,Full-time · Entry level,50 applicants,201-500 employees,"Technology, Information and Internet"
2,2,Data Analyst (OpEx Engineering),"Mumbai, Maharashtra, India",Fynd (Shopsense Retail Technologies Ltd.),Full-time · Entry level,39 applicants,"501-1,000 employees",Internet Marketplace Platforms
3,3,UCC Quality Monitoring Developer/Analyst,"Bengaluru, Karnataka, India",Bosch Global Software Technologies,Full-time · Associate,65 applicants,"10,001+ employees",IT Services and IT Consulting
4,4,Associate Systems Analyst,"Pune, Maharashtra, India",Allscripts India,Full-time · Entry level,52 applicants,"1,001-5,000 employees",IT Services and IT Consulting
...,...,...,...,...,...,...,...,...
456,456,Product Marketing - General Manager,"Noida, Uttar Pradesh, India",Paytm,Full-time · Executive,200 applicants,"10,001+ employees",Financial Services
457,457,Principal Software Dev Engr,"Bengaluru, Karnataka, India",Tesco Technology,Full-time,10 applicants,"1,001-5,000 employees",Retail
458,458,Associate Consultant,"Noida, Uttar Pradesh, India",Huron,Full-time,75 applicants,"1,001-5,000 employees",Business Consulting and Services
459,459,Simulation Engineering - SPDM,"Bengaluru, Karnataka, India",Bosch Global Software Technologies,Full-time · Associate,36 applicants,"10,001+ employees",IT Services and IT Consulting


In [54]:
df2.isnull().value_counts()

Unnamed: 0  designation  location  company_name  involvement  total_applicants  employees_count  industry_type
False       False        False     False         False        False             False            False            403
dtype: int64

In [55]:
df2.drop(["Unnamed: 0"],axis=1,inplace=True)

In [56]:
df2

,designation,location,company_name,involvement,total_applicants,employees_count,industry_type
0,Data Analyst For 5 Day Working USA MNC In Gurg...,"Gurgaon, Haryana, India",Capital Placement Services Gurgaon .,Full-time · Entry level,34 applicants,1-10 employees,Human Resources Services
1,Zoomcar - Program Manager - Process Excellence,"Bengaluru, Karnataka, India",Zoomcar,Full-time · Entry level,50 applicants,201-500 employees,"Technology, Information and Internet"
2,Data Analyst (OpEx Engineering),"Mumbai, Maharashtra, India",Fynd (Shopsense Retail Technologies Ltd.),Full-time · Entry level,39 applicants,"501-1,000 employees",Internet Marketplace Platforms
3,UCC Quality Monitoring Developer/Analyst,"Bengaluru, Karnataka, India",Bosch Global Software Technologies,Full-time · Associate,65 applicants,"10,001+ employees",IT Services and IT Consulting
4,Associate Systems Analyst,"Pune, Maharashtra, India",Allscripts India,Full-time · Entry level,52 applicants,"1,001-5,000 employees",IT Services and IT Consulting
...,...,...,...,...,...,...,...
456,Product Marketing - General Manager,"Noida, Uttar Pradesh, India",Paytm,Full-time · Executive,200 applicants,"10,001+ employees",Financial Services
457,Principal Software Dev Engr,"Bengaluru, Karnataka, India",Tesco Technology,Full-time,10 applicants,"1,001-5,000 employees",Retail
458,Associate Consultant,"Noida, Uttar Pradesh, India",Huron,Full-time,75 applicants,"1,001-5,000 employees",Business Consulting and Services
459,Simulation Engineering - SPDM,"Bengaluru, Karnataka, India",Bosch Global Software Technologies,Full-time · Associate,36 applicants,"10,001+ employees",IT Services and IT Consulting


In [57]:
Details = df2[["involvement","total_applicants"]]

In [64]:
Details.to_csv("Final_details_table.csv",index=False)

In [61]:
Job = df2[["designation","location","company_name"]]

In [65]:
Job.to_csv("Final_job_table.csv",index=False)

In [66]:
df2.to_csv("Final_Linkedin_data.csv",index=False)